<a href="https://colab.research.google.com/github/Harshitmaury123/Machine_Learning_Lab/blob/main/ML_LAB_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
df=pd.read_csv('faa_ai_prelim.csv')
df.head()

,UPDATED,ENTRY_DATE,EVENT_LCL_DATE,EVENT_LCL_TIME,LOC_CITY_NAME,LOC_STATE_NAME,LOC_CNTRY_NAME,RMK_TEXT,EVENT_TYPE_DESC,FSDO_DESC,...,PAX_INJ_NONE,PAX_INJ_MINOR,PAX_INJ_SERIOUS,PAX_INJ_FATAL,PAX_INJ_UNK,GRND_INJ_NONE,GRND_INJ_MINOR,GRND_INJ_SERIOUS,GRND_INJ_FATAL,GRND_INJ_UNK
0,No,19-FEB-16,19-FEB-16,00:45:00Z,MARSHVILLE,North Carolina,NaN,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",Accident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No,19-FEB-16,18-FEB-16,23:55:00Z,TAVERNIER,Florida,NaN,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,Incident,FAA Miami FSDO-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,No,19-FEB-16,18-FEB-16,22:14:00Z,TRENTON,New Jersey,NaN,"AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...",Incident,FAA Philadelphia FSDO-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,19-FEB-16,18-FEB-16,17:10:00Z,ASHEVILLE,North Carolina,NaN,"AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...",Incident,FAA Charlotte FSDO-68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,No,19-FEB-16,18-FEB-16,00:26:00Z,TALKEETNA,Alaska,NaN,"AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...",Incident,FAA Anchorage FSDO-03,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UPDATED              83 non-null     object 
 1   ENTRY_DATE           83 non-null     object 
 2   EVENT_LCL_DATE       83 non-null     object 
 3   EVENT_LCL_TIME       83 non-null     object 
 4   LOC_CITY_NAME        83 non-null     object 
 5   LOC_STATE_NAME       83 non-null     object 
 6   LOC_CNTRY_NAME       2 non-null      object 
 7   RMK_TEXT             83 non-null     object 
 8   EVENT_TYPE_DESC      83 non-null     object 
 9   FSDO_DESC            81 non-null     object 
 10  REGIST_NBR           83 non-null     object 
 11  FLT_NBR              6 non-null      object 
 12  ACFT_OPRTR           7 non-null      object 
 13  ACFT_MAKE_NAME       78 non-null     object 
 14  ACFT_MODEL_NAME      79 non-null     object 
 15  ACFT_MISSING_FLAG    0 non-null      float

In [23]:
# Convert date and time columns to a single datetime object
df['EVENT_DATETIME'] = pd.to_datetime(df['EVENT_LCL_DATE'] + ' ' + df['EVENT_LCL_TIME'], errors='coerce')

# Extract required attributes
required_columns = [
    'ACFT_MAKE_NAME',
    'LOC_STATE_NAME',
    'ACFT_MODEL_NAME',
    'RMK_TEXT',
    'FLT_PHASE',
    'EVENT_TYPE_DESC',
    'FATAL_FLAG',
    'EVENT_DATETIME', # Include the new datetime column
    'ACFT_DMG_DESC' # Include the ACFT_DMG_DESC column
]

# Check if ACFT_DMG_DESC is in the original dataframe columns
if 'ACFT_DMG_DESC' not in df.columns:
    print("Error: 'ACFT_DMG_DESC' column not found in the original dataframe.")
else:
    # Create a new dataframe with only the required columns
    df_processed = df[required_columns].copy()

    # Replace missing Fatal Flag values with 'No'
    df_processed['FATAL_FLAG'] = df_processed['FATAL_FLAG'].fillna('No')

    # Fill missing values in FLT_PHASE and ACFT_DMG_DESC with the mode
    df_processed['FLT_PHASE'] = df_processed['FLT_PHASE'].fillna(df_processed['FLT_PHASE'].mode()[0])
    df_processed['ACFT_DMG_DESC'] = df_processed['ACFT_DMG_DESC'].fillna(df_processed['ACFT_DMG_DESC'].mode()[0])


    # Verify if missing values are replaced
    print("Missing values after filling:")
    print(df_processed.isnull().sum())

    # Check the number of observations
    print("\nNumber of observations before dropping rows:", len(df_processed))

    # Drop rows where aircraft names are not available
    df_processed.dropna(subset=['ACFT_MAKE_NAME'], inplace=True)

    # Check the number of observations after dropping rows
    print("\nNumber of observations after dropping rows:", len(df_processed))


    # Group the dataset by aircraft name
    aircraft_counts = df_processed['ACFT_MAKE_NAME'].value_counts()

    # View the number of times each aircraft type appears in the dataset
    print("\nNumber of times each aircraft type appears:")
    print(aircraft_counts)

    # Display the observations where fatal flag is "Yes"
    fatal_incidents = df_processed[df_processed['FATAL_FLAG'] == 'Yes']
    print("\nObservations where fatal flag is 'Yes':")
    display(fatal_incidents)

    # Print columns of df_processed before one-hot encoding
    print("\nColumns of df_processed before one-hot encoding:")
    print(df_processed.columns)

    # Perform one-hot encoding on ACFT_DMG_DESC
    df_processed = pd.get_dummies(df_processed, columns=['ACFT_DMG_DESC'], drop_first=True)

    # Display the first few rows of the processed dataframe with one-hot encoding
    print("\nProcessed dataframe after one-hot encoding:")
    display(df_processed.head())

Missing values after filling:
ACFT_MAKE_NAME     5
LOC_STATE_NAME     0
ACFT_MODEL_NAME    4
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
EVENT_DATETIME     0
ACFT_DMG_DESC      0
dtype: int64

Number of observations before dropping rows: 83

Number of observations after dropping rows: 78

Number of times each aircraft type appears:
ACFT_MAKE_NAME
CESSNA                    23
PIPER                     10
BEECH                      9
MOONEY                     4
BOEING                     3
BELL                       2
SOCATA                     2
LANCAIR                    2
CHAMPION                   2
VANS                       1
GLOBE                      1
GREAT LAKES                1
HUGHES                     1
CHRISTEN                   1
FAIRCHILD                  1
CONSOLIDATED VULTEE        1
SAAB                       1
EMBRAER                    1
ENSTROM                    1
MAULE                      1
PITTS                      1
AE

/tmp/ipython-input-2223555133.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['EVENT_DATETIME'] = pd.to_datetime(df['EVENT_LCL_DATE'] + ' ' + df['EVENT_LCL_TIME'], errors='coerce')


,ACFT_MAKE_NAME,LOC_STATE_NAME,ACFT_MODEL_NAME,RMK_TEXT,FLT_PHASE,EVENT_TYPE_DESC,FATAL_FLAG,EVENT_DATETIME,ACFT_DMG_DESC
0,BEECH,North Carolina,36,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",UNKNOWN (UNK),Accident,Yes,2016-02-19 00:45:00+00:00,Unknown
53,PIPER,Florida,PA28,AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...,UNKNOWN (UNK),Accident,Yes,2016-02-11 11:32:00+00:00,Destroyed
55,FLIGHT DESIGN,California,CTLS,AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...,UNKNOWN (UNK),Accident,Yes,2016-02-11 00:17:00+00:00,Destroyed
79,NORTH AMERICAN,Arizona,F51,"AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...",UNKNOWN (UNK),Accident,Yes,2016-02-05 11:17:00+00:00,Destroyed
80,CHAMPION,California,8KCAB,"N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...",UNKNOWN (UNK),Accident,Yes,2016-02-05 23:02:00+00:00,Destroyed
81,BEECH,California,35,"N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...",UNKNOWN (UNK),Accident,Yes,2016-02-05 23:02:00+00:00,Destroyed
82,CESSNA,Alabama,182,N784CP AIRCRAFT CRASHED INTO A WOODED AREA NEA...,UNKNOWN (UNK),Accident,Yes,2016-02-02 01:52:00+00:00,Unknown



Columns of df_processed before one-hot encoding:
Index(['ACFT_MAKE_NAME', 'LOC_STATE_NAME', 'ACFT_MODEL_NAME', 'RMK_TEXT',
       'FLT_PHASE', 'EVENT_TYPE_DESC', 'FATAL_FLAG', 'EVENT_DATETIME',
       'ACFT_DMG_DESC'],
      dtype='object')

Processed dataframe after one-hot encoding:


,ACFT_MAKE_NAME,LOC_STATE_NAME,ACFT_MODEL_NAME,RMK_TEXT,FLT_PHASE,EVENT_TYPE_DESC,FATAL_FLAG,EVENT_DATETIME,ACFT_DMG_DESC_Minor,ACFT_DMG_DESC_Substantial,ACFT_DMG_DESC_Unknown
0,BEECH,North Carolina,36,"AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...",UNKNOWN (UNK),Accident,Yes,2016-02-19 00:45:00+00:00,False,False,True
1,VANS,Florida,RV7,AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...,LANDING (LDG),Incident,No,2016-02-18 23:55:00+00:00,False,False,True
2,CESSNA,New Jersey,172,"AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...",APPROACH (APR),Incident,No,2016-02-18 22:14:00+00:00,True,False,False
3,LANCAIR,North Carolina,235,"AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...",LANDING (LDG),Incident,No,2016-02-18 17:10:00+00:00,False,False,True
4,CESSNA,Alaska,172,"AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...",LANDING (LDG),Incident,No,2016-02-18 00:26:00+00:00,True,False,False
